# MIMIC III Preprocessing

* Input:
  * ./data/NOTEEVENTS-2.csv (contains clinical notes / text)
  * ./data/DIAGNOSES_ICD.csv (contains admission to ICD9 code diagnosis)
  * ./data/D_ICD_DIAGNOSES.csv (contains the ICD9 code description)
* Output:
  * ./data/DATA_HADM (contains top 50 icd9code, top 50 icd9cat, and clinical text for each admission)
  * ./data/DATA_HADM_CLEANED (contains top 50 icd9code, top 50 icd9cat, and cleaned clinical text w/out stopwords for each admission)
  * ./data/ICD9CODES.p (pickle file of all ICD9 codes)
  * ./data/ICD9CODES_TOP10.p (pickle file of top 10 ICD9 codes)
  * ./data/ICD9CODES_TOP50.p (pickle file of top 50 ICD9 codes)
  * ./data/ICD9CAT_TOP10.p (pickle file of top 10 ICD9 categories)
  * ./data/ICD9CAT_TOP50.p (pickle file of top 50 ICD9 categories)
* Description: converts the raw MIMIC III csv files into multi-labels-and-texts csv format. The output of this notebook are dumped in corresponding folders, which is merged by merge_data.sh

## Initialization and Data Loading

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
import functools
import time
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

conf = SparkConf().setAppName("preprocess").setMaster("local")
sc = SparkContext.getOrCreate(conf)
spark = SparkSession.builder.master("local").appName("preprocess").getOrCreate()

In [3]:
df_ne = pd.read_csv("../data/NOTEEVENTS-2.csv")

In [4]:
df_ne = df_ne[df_ne["category"] == "Discharge summary"]
df_ne.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59652 entries, 0 to 2083179
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   row_id       59652 non-null  int64  
 1   subject_id   59652 non-null  int64  
 2   hadm_id      59652 non-null  float64
 3   chartdate    59652 non-null  object 
 4   category     59652 non-null  object 
 5   description  59652 non-null  object 
 6   cgid         0 non-null      float64
 7   iserror      0 non-null      float64
 8   text         59652 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 4.6+ MB


In [5]:
# many icd to one hadm_id
df_diag_m = pd.read_csv("../data/DIAGNOSES_ICD.csv")

lower = {col: col.lower() for col in df_diag_m.columns}
df_diag_m.rename(columns=lower, inplace=True)

# added to filter out categories
df_diag_m["icd9_cat"] = df_diag_m["icd9_code"].apply(lambda x: 'c'+str(x)[:3])
# df_diag_m.head()

# one icd to one hadm_id (take the smallest seq number as primary)
keys = ["hadm_id", "subject_id", "seq_num"]
df_diag_o = df_diag_m.sort_values(keys).groupby("hadm_id").first().reset_index()


# get hadm_id list in noteevents
df_hadm_id_list = df_ne["hadm_id"].drop_duplicates()
# get subject_id list in noteevents
df_subject_id_list = df_ne["subject_id"].drop_duplicates()

df_icd9desc = pd.read_csv("../data/D_ICD_DIAGNOSES.csv")
df_icd9desc.head()

#filter out diag has no hopistal stay data
df_diag_o2 = df_diag_o[df_diag_o["hadm_id"].isin(df_hadm_id_list)]
df_diag_m2 = df_diag_m[df_diag_m["hadm_id"].isin(df_hadm_id_list)]

In [6]:
# data = [tuple(x) for x in df_diag_m.sort_values(keys).head(30).values]
# df = spark.createDataFrame(data,["row_id", "subject_id", "hadm_id", "seq_num", "icd9_code","icd9_cat"])
# df.show()

# diag_o_rdd = df.rdd.sortBy(lambda x: (x.hadm_id, x.subject_id, x.seq_num)) \
#     .groupBy(lambda x: x.hadm_id) \
#     .mapValues(list) \
#     .reduceByKey(lambda x, y: x if x.seq_num < y.seq_num else y) \
#     .map(lambda hid_d: hid_d[1][0])

# diag_o_rdd.collect()

In [7]:
print("Discharge summary: ", df_ne.shape)
print("Diagnosis codes: ", df_diag_m.shape)
print("Diagnosis codes require one code one stay: ", df_diag_o.shape)
print("Unique hadm_id: ",df_hadm_id_list.shape)
print("Unique subject_id: ", df_subject_id_list.shape)
print("Diag dictionary: ", df_icd9desc.shape)
print(df_diag_o2.shape)
print(df_diag_m2.shape)

Discharge summary:  (59652, 9)
Diagnosis codes:  (651047, 6)
Diagnosis codes require one code one stay:  (58976, 6)
Unique hadm_id:  (52726,)
Unique subject_id:  (41127,)
Diag dictionary:  (14567, 4)
(52726, 6)
(619002, 6)


## Data Preprocessing (all icd9 codes)

Returns RDD[(hadm_id, list(icd9_codes))]

In [8]:
icd9code_score_hadm = df_diag_m2.groupby("icd9_code")["hadm_id"].apply(lambda x: len(x.unique())).reset_index().rename(columns={"hadm_id": "score"})
icd9code_score_subj = df_diag_m2.groupby("icd9_code")["subject_id"].apply(lambda x: len(x.unique())).reset_index().rename(columns={"subject_id": "score"})

icd9cat_score_hadm = df_diag_m2.groupby("icd9_cat")["hadm_id"].apply(lambda x: len(x.unique())).reset_index().rename(columns={"hadm_id": "score"})
icd9cat_score_subj = df_diag_m2.groupby("icd9_cat")["subject_id"].apply(lambda x: len(x.unique())).reset_index().rename(columns={"subject_id": "score"})

In [9]:
icd9cat_score_subj

,icd9_cat,score
0,c003,6
1,c004,5
2,c005,4
3,c007,5
4,c008,1438
...,...,...
937,cV87,368
938,cV88,149
939,cV90,6
940,cV91,1


In [10]:
def get_id_to_topicd9(id_type, icdcode, topX):
    """
    Return: 
        id_to_topicd9: pandas dataframe(id, icd9)
        icd9_topX2: list of top icd9 code or category
    """
    if id_type == "hadm_id" and icdcode:
        icd9_score = icd9code_score_hadm
    elif id_type == "hadm_id" and not icdcode:
        icd9_score = icd9cat_score_hadm
    elif id_type == "subject_id" and icdcode:
        icd9_score = icd9code_score_subj
    elif id_type == "subject_id" and not icdcode:
        icd9_score = icd9cat_score_subj
    else: #default
        icd9_score = icd9code_score_hadm
    
    if not icdcode:
        icd9_topX2 = icd9_score.sort_values("score", ascending=False)["icd9_cat"].head(topX).tolist()
        icd9_topX2 = [str(i) for i in icd9_topX2]
#         print(icd9_topX2)
    else:
        icd9_topX2 = icd9_score.sort_values("score", ascending=False)["icd9_code"].head(topX).tolist()
        icd9_topX2 = [str(i) for i in icd9_topX2]
    icd9_topX = set(icd9_topX2)
    
    if icdcode:
        id_to_topicd9 = df_diag_m2[df_diag_m2["icd9_code"].isin(icd9_topX)].groupby(id_type)["icd9_code"].apply(set).reset_index()
    else:
        id_to_topicd9 = df_diag_m2[df_diag_m2["icd9_cat"].isin(icd9_topX)].groupby(id_type)["icd9_cat"].apply(set).reset_index()
        
    return id_to_topicd9, icd9_topX2

In [11]:
# icd9_topX = set(icd9code_score_hadm.sort_values("score", ascending=False).head(50)["icd9_code"].tolist())

# data = [tuple(x) for x in df_diag_m2.head(500).values]
# df = spark.createDataFrame(data,["row_id", "subject_id", "hadm_id", "seq_num", "icd9_code", "icd9_cat"])
# # df.show()

# id_to_topicd9 = df.rdd \
#         .map(lambda x: (x.hadm_id, x.icd9_code)) \
#         .groupByKey() \
#         .mapValues(lambda x: set(x) & icd9_topX) \
#         .filter(lambda xy: xy[1])

# id_to_topicd9.collect()

In [19]:
topX = 10
print(get_id_to_topicd9("hadm_id", True, topX)[0].shape)
print(get_id_to_topicd9("subject_id", True, topX)[0].shape)
print(get_id_to_topicd9("hadm_id", False, topX)[0].shape)
print(get_id_to_topicd9("subject_id", False, topX)[0].shape)

(40562, 2)
(30959, 2)
(44419, 2)
(33931, 2)


In [20]:
id_type, topX = "hadm_id", 10
id_to_topicd9code, topicd9code = get_id_to_topicd9(id_type, True, topX)
id_to_topicd9cat, topicd9cat = get_id_to_topicd9(id_type, False, topX)

In [21]:
import re

def sparse2vec(mapper, data):
    out = [0] * len(mapper)
    if data != None:
        for i in data:
            out[mapper[i]] = 1
    return out

def combine_set(row):
#     print(row)
    if type(row["icd9_cat"]) != set:
        return row["icd9_code"]
    elif type(row["icd9_code"]) != set:
        return row["icd9_cat"]
    else:
        return row["icd9_code"] | row["icd9_cat"]
    
def get_id_to_texticd9(id_type, topX, icd9type="both", stopwords=[]):
    def remstopwords(text):
        text = re.sub('\[\*\*[^\]]*\*\*\]', '', text)
        text = re.sub('<[^>]*>', '', text)
        text = re.sub('[\W]+', ' ', text.lower()) 
        text = re.sub(" \d+", " ", text)
        return " ".join([i for i in text.split() if i not in stopwords])
    
    if icd9type == "both":
        id_to_topicd9code, topicd9code = get_id_to_topicd9(id_type, True, topX)
        id_to_topicd9cat, topicd9cat = get_id_to_topicd9(id_type, False, topX)
        topX2 = 2 * topX
        topicd9 = topicd9code+topicd9cat
        mapper = dict(zip(topicd9, range(topX2)))
        
        id_to_topicd9 = id_to_topicd9code.merge(id_to_topicd9cat, how="outer", on=id_type)
        id_to_topicd9["icd9"] = id_to_topicd9.apply(axis=1, func=combine_set)
    elif icd9type == "icd9code":
        id_to_topicd9, topicd9code = get_id_to_topicd9(id_type, True, topX)
        id_to_topicd9.rename(columns={"icd9_code":"icd9"}, inplace=True)
        topicd9 = topicd9code
        mapper = dict(zip(topicd9, range(topX)))
    else:
        id_to_topicd9, topicd9cat = get_id_to_topicd9(id_type, False, topX)
        id_to_topicd9.rename(columns={"icd9_cat":"icd9"}, inplace=True)
        topicd9 = topicd9cat
        mapper = dict(zip(topicd9, range(topX)))
    
    
    vectorize = functools.partial(sparse2vec, mapper)
    id_to_topicd9 = id_to_topicd9.apply(axis=1, func=lambda row: [row[id_type]] + vectorize(row["icd9"]), result_type='expand')
    id_to_topicd9.rename(columns={0:id_type}, inplace=True)
    
    df_ne_2 = df_ne.groupby(id_type)["text"].apply(lambda x: "".join(x)).reset_index()
    df_ne_2["text"] = df_ne_2["text"].apply(lambda x: x if len(stopwords)==0 else remstopwords(x))
    
    data = df_ne_2.merge(id_to_topicd9, how="left", on=id_type)
    data.fillna(0, inplace=True) #some summary doesnot contain topK icd9
    
    colnames = dict(zip(range(1, len(topicd9)+1), topicd9))
    colnames[id_type]="id"
    dtypes = dict(zip(colnames.keys(), [np.int32]*len(colnames)))

    data = data.astype(dtypes, copy=True)
    data.rename(columns=colnames, inplace=True)
    
    return data, topicd9

Make list of unique ICD9CODES

In [22]:
ICD9CODES = [str(i).lower() for i in df_diag_m2["icd9_code"].unique()]
# pickle.dump(ICD9CODES, open( "../data/ICD9CODES.p", "wb" ))

In [30]:
import time
t0 = time.time()

df_id2texticd9, topicd9 = get_id_to_texticd9("hadm_id", 50)
# df_id2texticd9.to_csv("../data/DATA_HADM_top10_icd9code.csv", header=True, index=False)

print(topicd9)
print(df_id2texticd9.count())
print(time.time() - t0)

df_id2texticd9.head()

['4019', '4280', '42731', '41401', '5849', '25000', '2724', '51881', '5990', '53081', '2720', '2859', '2449', '486', '2851', '2762', '496', '99592', 'V5861', '5070', '0389', '5859', '40390', '311', '3051', '412', '2875', '41071', '2761', 'V4581', '4240', 'V1582', '5119', 'V4582', '40391', 'V290', '4241', '78552', 'V5867', '42789', '32723', '9971', '5845', '2760', '7742', '5180', 'V053', '4168', '49390', '2767', 'c401', 'c427', 'c276', 'c272', 'c414', 'c250', 'c428', 'c518', 'c285', 'c584', 'cV45', 'c599', 'c530', 'cV58', 'c585', 'cE87', 'c403', 'cV10', 'c038', 'c995', 'c424', 'c410', 'c780', 'c244', 'c997', 'c785', 'c305', 'c998', 'c458', 'c486', 'cV15', 'c041', 'c496', 'c996', 'c287', 'cV12', 'c790', 'c507', 'cE93', 'c493', 'c311', 'c511', 'c412', 'c707', 'c348', 'c765', 'cE88', 'c571', 'c300', 'c733']
id       52726
text     52726
4019     52726
4280     52726
42731    52726
         ...  
c765     52726
cE88     52726
c571     52726
c300     52726
c733     52726
Length: 102, dtype: 

,id,text,4019,4280,42731,41401,5849,25000,2724,51881,...,c311,c511,c412,c707,c348,c765,cE88,c571,c300,c733
0,100001,Admission Date: [**2117-9-11**] ...,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,100003,Admission Date: [**2150-4-17**] ...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,100006,Admission Date: [**2108-4-6**] Discharg...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,100007,Admission Date: [**2145-3-31**] ...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100009,Admission Date: [**2162-5-16**] ...,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
print(topicd9[:10])
pickle.dump(topicd9[:10], open( "../data/ICD9CODES_TOP10.p", "wb" ))
print(topicd9[:50])
pickle.dump(topicd9[:50], open( "../data/ICD9CODES_TOP50.p", "wb" ))
print(topicd9[50:60])
pickle.dump(topicd9[50:60], open( "../data/ICD9CAT_TOP10.p", "wb" ))
print(topicd9[50:])
pickle.dump(topicd9[50:], open( "../data/ICD9CAT_TOP50.p", "wb" ))

['4019', '4280', '42731', '41401', '5849', '25000', '2724', '51881', '5990', '53081']
['4019', '4280', '42731', '41401', '5849', '25000', '2724', '51881', '5990', '53081', '2720', '2859', '2449', '486', '2851', '2762', '496', '99592', 'V5861', '5070', '0389', '5859', '40390', '311', '3051', '412', '2875', '41071', '2761', 'V4581', '4240', 'V1582', '5119', 'V4582', '40391', 'V290', '4241', '78552', 'V5867', '42789', '32723', '9971', '5845', '2760', '7742', '5180', 'V053', '4168', '49390', '2767']
['c401', 'c427', 'c276', 'c272', 'c414', 'c250', 'c428', 'c518', 'c285', 'c584']
['c401', 'c427', 'c276', 'c272', 'c414', 'c250', 'c428', 'c518', 'c285', 'c584', 'cV45', 'c599', 'c530', 'cV58', 'c585', 'cE87', 'c403', 'cV10', 'c038', 'c995', 'c424', 'c410', 'c780', 'c244', 'c997', 'c785', 'c305', 'c998', 'c458', 'c486', 'cV15', 'c041', 'c496', 'c996', 'c287', 'cV12', 'c790', 'c507', 'cE93', 'c493', 'c311', 'c511', 'c412', 'c707', 'c348', 'c765', 'cE88', 'c571', 'c300', 'c733']


In [27]:
len(STOPWORDS_WORD2VEC)

7098

In [28]:
t0 = time.time()
STOPWORDS_WORD2VEC = stopwords.words('english') + ICD9CODES

df_id2texticd9, topicd9 = get_id_to_texticd9("hadm_id", 10, icd9type="icd9code", stopwords=set(STOPWORDS_WORD2VEC))
df_id2texticd9.to_csv("../data/DATA_HADM_CLEANED_top10_icd9code.csv", header=True, index=False)

print(topicd9)
print(df_id2texticd9.count())
print(time.time() - t0)
df_id2texticd9.head()

56.130059480667114


,id,text,4019,4280,42731,41401,5849,25000,2724,51881,5990,53081
0,100001,admission date discharge date date birth sex f...,0,0,0,0,1,0,0,0,0,0
1,100003,admission date discharge date date birth sex s...,1,0,0,0,0,0,0,0,0,0
2,100006,admission date discharge date date birth sex f...,0,0,0,0,0,0,0,1,0,0
3,100007,admission date discharge date date birth sex f...,1,0,0,0,0,0,0,0,0,0
4,100009,admission date discharge date date birth sex s...,1,0,0,1,0,1,0,0,0,0
